In [2]:
# import necessary library
import pandas as pd
import math
import seaborn as sns
from matplotlib import pyplot as plt
import re
import numpy as np
import geopandas as gpd
from shapely.geometry import Point # Point class
from shapely.geometry import shape
from shapely.geometry import Polygon, LineString, Point

C:\Users\Md Mintu Miah\anaconda3\envs\geo_env\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
# surpres all warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
# avoid unecessary zeros after interger year
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

In [5]:
# read CA strava geometry
CA_ge=gpd.read_file('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/strava_combined_2018_geometry/strava_combined_2018_geometry.shp')

In [6]:
CA_ge1=CA_ge.to_crs('epsg:4326')
CA_ge1.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
car_ge=gpd.read_file('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/selected_area_yearly_2018_ride_carson_city/fc2eba658781af8b2c78e649da0705cd5e3087f149abed4474a15598bd6f5e9f-1668731100883.shp')
CA_Carson_geo=pd.concat([CA_ge1,car_ge],axis=0)
CA_Carson_geo.set_index('edgeUID', inplace=True)
CA_Carson_geo.head()

,osmId,layer,path,geometry
edgeUID,,,,
308647639,8942624,075a1d3c2bcaa8641098036f74342333afbdaa711e1ae3...,D:/Data/SMART3/mine/Strava/2018/075a1d3c2bcaa8...,"LINESTRING (-122.49825 37.50955, -122.49821 37..."
308650445,425512742,075a1d3c2bcaa8641098036f74342333afbdaa711e1ae3...,D:/Data/SMART3/mine/Strava/2018/075a1d3c2bcaa8...,"LINESTRING (-122.41545 37.52096, -122.41521 37..."
308656199,8936603,075a1d3c2bcaa8641098036f74342333afbdaa711e1ae3...,D:/Data/SMART3/mine/Strava/2018/075a1d3c2bcaa8...,"LINESTRING (-122.39397 37.57525, -122.39446 37..."
308657506,586554300,075a1d3c2bcaa8641098036f74342333afbdaa711e1ae3...,D:/Data/SMART3/mine/Strava/2018/075a1d3c2bcaa8...,"LINESTRING (-122.39911 37.60547, -122.39923 37..."
308657891,396965439,075a1d3c2bcaa8641098036f74342333afbdaa711e1ae3...,D:/Data/SMART3/mine/Strava/2018/075a1d3c2bcaa8...,"LINESTRING (-122.39722 37.60514, -122.39738 37..."


In [8]:
CA_Carson_geo.shape,CA_Carson_geo['osmId'].nunique()

((7074662, 4), 3065170)

In [9]:
CA_Carson_geo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
# crs = {'init': 'epsg:4326'}
# gdf_geo = gpd.GeoDataFrame(CA_Carson_geo, crs=crs, geometry = 'geometry')
# gdf_geo.to_file("C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/GIS File/CA_Carson_Strava_Annual_Geometry_2018.shp")

In [10]:
# Match the short term sites with strava
# read short term sites from 2014 to 2019
sc=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Processed Data/short_term_clean_data_2014_2019_unique_location.csv')
sc.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ID,Date,Day of Week,Start Time,End Time,Count,Leg,...,dayofweek,weekday_Status,Week Day Status,starttime,endtime,starttime1,endtime1,startDateTime,endDateTime,diff
0,3432,3432,0,CH0,2018-05-30,Wednesday,7:00 AM,8:00 AM,4,N,...,2.00,0,Week Day,07:00:00,08:00:00,NaN,NaN,2018-05-30 07:00:00,2018-05-30 08:00:00,1.00
1,3438,3438,6,CH0,2018-05-30,Wednesday,7:00 AM,8:00 AM,1,E,...,2.00,0,Week Day,07:00:00,08:00:00,NaN,NaN,2018-05-30 07:00:00,2018-05-30 08:00:00,1.00
2,3442,3442,10,CH0,2018-05-30,Wednesday,7:00 AM,8:00 AM,3,S,...,2.00,0,Week Day,07:00:00,08:00:00,NaN,NaN,2018-05-30 07:00:00,2018-05-30 08:00:00,1.00
3,3451,3451,19,CH1,2018-05-30,Wednesday,7:00 AM,8:00 AM,1,N,...,2.00,0,Week Day,07:00:00,08:00:00,NaN,NaN,2018-05-30 07:00:00,2018-05-30 08:00:00,1.00
4,3457,3457,25,CH1,2018-05-30,Wednesday,8:00 AM,9:00 AM,1,E,...,2.00,0,Week Day,08:00:00,09:00:00,NaN,NaN,2018-05-30 08:00:00,2018-05-30 09:00:00,1.00


In [11]:
sc.dtypes

Unnamed: 0.2         int64
Unnamed: 0.1         int64
Unnamed: 0           int64
ID                  object
Date                object
Day of Week         object
Start Time          object
End Time            object
Count                int64
Leg                 object
matched_seg_id       int64
date                object
month              float64
year               float64
day                float64
dayofweek          float64
weekday_Status       int64
Week Day Status     object
starttime           object
endtime             object
starttime1          object
endtime1            object
startDateTime       object
endDateTime         object
diff               float64
dtype: object

In [12]:
# Read the SHS segment geometry
shs=gpd.read_file('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Independent Variables/SHS shape file/smart4_final_output_Feb2023.shp')
shs.head()

,segment_id,street_nam,county,tdg_id,lrs_cal_id,bikes_proh,bike_facs,int_tdg_id,loc_id,seg_counte,...,rt_lanes_a,lt_lanes_a,rt_trav_wa,rt_i_shd_t,rt_o_shd_t,lt_trav_wa,lt_i_shd_t,lt_o_shd_t,Shape_Leng,geometry
0,None,Emerald Bay Road,El Dorado,8041.00,0,0,None,None,None,None,...,1.00,1.00,12.00,0.00,2.00,12.00,0.00,2.00,5416.42,"LINESTRING Z (752783.636 4315141.964 0.000, 75..."
1,82664,Patrick Creek Road,Del Norte,7027.00,0,0,None,None,None,None,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4715.91,"LINESTRING Z (429129.958 4640105.752 0.000, 42..."
2,None,Hopland Road,Lake,30797.00,637,0,None,None,None,None,...,1.00,1.00,10.00,0.00,2.00,10.00,0.00,2.00,1972.57,"LINESTRING Z (501148.889 4315538.675 0.000, 50..."
3,59913,Cannon Road,Tehama,129091.00,0,0,None,None,None,None,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6360.20,"LINESTRING Z (541160.263 4449715.440 0.000, 54..."
4,107762,RAMP,San Bernardino,96317.00,0,0,None,None,None,None,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,187.05,"LINESTRING Z (1015827.620 3801100.883 0.000, 1..."


In [13]:
shs1=shs.to_crs('epsg:4326')
shs1.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [14]:
shs.crs

<Derived Projected CRS: EPSG:26910>
Name: NAD83 / UTM zone 10N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: North America - between 126°W and 120°W - onshore and offshore. Canada - British Columbia; Northwest Territories; Yukon. United States (USA) - California; Oregon; Washington.
- bounds: (-126.0, 30.54, -119.99, 81.8)
Coordinate Operation:
- name: UTM zone 10N
- method: Transverse Mercator
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [15]:
shs.dtypes

segment_id      object
street_nam      object
county          object
tdg_id         float64
lrs_cal_id      object
                ...   
lt_trav_wa     float64
lt_i_shd_t     float64
lt_o_shd_t     float64
Shape_Leng     float64
geometry      geometry
Length: 167, dtype: object

In [16]:
shs.shape, sc.shape

((87154, 167), (6816, 25))

In [17]:
# lets match short term sites with its geometry
sc_shs=pd.merge(sc, shs1, left_on=['matched_seg_id'], right_on=['tdg_id'], how='inner')
sc_shs1 = gpd.GeoDataFrame(sc_shs, crs="EPSG:4326", geometry='geometry')
sc_shs1.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ID,Date,Day of Week,Start Time,End Time,Count,Leg,...,rt_lanes_a,lt_lanes_a,rt_trav_wa,rt_i_shd_t,rt_o_shd_t,lt_trav_wa,lt_i_shd_t,lt_o_shd_t,Shape_Leng,geometry
0,3432,3432,0,CH0,2018-05-30,Wednesday,7:00 AM,8:00 AM,4,N,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,202.35,"LINESTRING (-121.29043 38.72027, -121.29043 38..."
1,3438,3438,6,CH0,2018-05-30,Wednesday,7:00 AM,8:00 AM,1,E,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1623.51,"LINESTRING (-121.29043 38.71845, -121.29023 38..."
2,3442,3442,10,CH0,2018-05-30,Wednesday,7:00 AM,8:00 AM,3,S,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,104.94,"LINESTRING (-121.29043 38.71845, -121.29043 38..."
3,3451,3451,19,CH1,2018-05-30,Wednesday,7:00 AM,8:00 AM,1,N,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,95.07,"LINESTRING (-121.29039 38.70747, -121.29040 38..."
4,3461,3461,29,CH1,2018-05-30,Wednesday,7:00 AM,8:00 AM,1,S,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,205.17,"LINESTRING (-121.29040 38.70661, -121.29040 38..."


In [18]:
sc_shs1['Center_point'] = sc_shs1['geometry'].centroid
sc_shs1.head()

C:\Users\Md Mintu Miah\AppData\Local\Temp\ipykernel_19468\3068556460.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  sc_shs1['Center_point'] = sc_shs1['geometry'].centroid


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ID,Date,Day of Week,Start Time,End Time,Count,Leg,...,lt_lanes_a,rt_trav_wa,rt_i_shd_t,rt_o_shd_t,lt_trav_wa,lt_i_shd_t,lt_o_shd_t,Shape_Leng,geometry,Center_point
0,3432,3432,0,CH0,2018-05-30,Wednesday,7:00 AM,8:00 AM,4,N,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,202.35,"LINESTRING (-121.29043 38.72027, -121.29043 38...",POINT (-121.29043 38.71936)
1,3438,3438,6,CH0,2018-05-30,Wednesday,7:00 AM,8:00 AM,1,E,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1623.51,"LINESTRING (-121.29043 38.71845, -121.29023 38...",POINT (-121.28109 38.71851)
2,3442,3442,10,CH0,2018-05-30,Wednesday,7:00 AM,8:00 AM,3,S,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,104.94,"LINESTRING (-121.29043 38.71845, -121.29043 38...",POINT (-121.29043 38.71797)
3,3451,3451,19,CH1,2018-05-30,Wednesday,7:00 AM,8:00 AM,1,N,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,95.07,"LINESTRING (-121.29039 38.70747, -121.29040 38...",POINT (-121.29040 38.70704)
4,3461,3461,29,CH1,2018-05-30,Wednesday,7:00 AM,8:00 AM,1,S,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,205.17,"LINESTRING (-121.29040 38.70661, -121.29040 38...",POINT (-121.29040 38.70569)


In [19]:
sc_shs2=sc_shs1.drop(['geometry'], axis=1)
sc_shs3=sc_shs2.rename(columns={'Center_point':'geometry'})
sc_shs3.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ID,Date,Day of Week,Start Time,End Time,Count,Leg,...,rt_lanes_a,lt_lanes_a,rt_trav_wa,rt_i_shd_t,rt_o_shd_t,lt_trav_wa,lt_i_shd_t,lt_o_shd_t,Shape_Leng,geometry
0,3432,3432,0,CH0,2018-05-30,Wednesday,7:00 AM,8:00 AM,4,N,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,202.35,POINT (-121.29043 38.71936)
1,3438,3438,6,CH0,2018-05-30,Wednesday,7:00 AM,8:00 AM,1,E,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1623.51,POINT (-121.28109 38.71851)
2,3442,3442,10,CH0,2018-05-30,Wednesday,7:00 AM,8:00 AM,3,S,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,104.94,POINT (-121.29043 38.71797)
3,3451,3451,19,CH1,2018-05-30,Wednesday,7:00 AM,8:00 AM,1,N,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,95.07,POINT (-121.29040 38.70704)
4,3461,3461,29,CH1,2018-05-30,Wednesday,7:00 AM,8:00 AM,1,S,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,205.17,POINT (-121.29040 38.70569)


In [20]:
# crs = {'init': 'epsg:4326'}
# sc_shs4 = gpd.GeoDataFrame(sc_shs3, crs=crs, geometry = 'geometry')
# sc_shs4.to_file("C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/GIS File/short_term_sites_geom.shp")

C:\Users\Md Mintu Miah\AppData\Local\Temp\ipykernel_19468\3762004464.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  sc_shs4.to_file("C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/GIS File/short_term_sites_geom.shp")


In [54]:
sc_shs3.shape

(5711, 192)

In [1]:
# # Match with Strava
# def nearest_linestring(point, df_lines):
#     idx = df_lines.geometry.distance(point).idxmin()
#     return df_lines.loc[idx, 'osmId']

# sc_shs3['osmId '] = sc_shs3.geometry.apply(nearest_linestring, df_lines=CA_Carson_geo)
# sc_shs3.head()

In [48]:
# read eco_strava matched data
sc_stv=gpd.read_file('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Processed Data/Short_term_strava_match_Feb7.shp')
sc_stv=sc_stv.drop([ 'Unnamed__0', 'Unnamed__1', 'matched_se','Unnamed__2','OBJECTID','Join_Count', 'TARGET_FID','path','Date', 'Day_of_Wee', 'Start_Time', 'End_Time', 'Count', 'month','day', 'dayofweek',
       'weekday_St', 'Week_Day_S', 'starttime', 'endtime', 'starttime1',
       'endtime1', 'startDateT', 'endDateTim', 'diff','layer'],axis=1)
sc_stv.head()

,ID,Leg,date_1,year,segment_id,street_nam,county,tdg_id,lrs_cal_id,bikes_proh,...,rt_trav_wa,rt_i_shd_t,rt_o_shd_t,lt_trav_wa,lt_i_shd_t,lt_o_shd_t,Shape_Leng,edgeUID,osmId,geometry
0,CH0,N,2018-05-30,2018.00,148204,Auburn Boulevard,Sacramento,90150.00,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,202.35,310765843.00,502900510.00,POINT (-121.29043 38.71936)
1,CH0,E,2018-05-30,2018.00,143632,Twin Oaks Avenue,Sacramento,89862.00,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,1623.51,310802215.00,586450892.00,POINT (-121.28109 38.71851)
2,CH0,S,2018-05-30,2018.00,141109,Auburn Boulevard,Sacramento,89555.00,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,104.94,310765852.00,617513985.00,POINT (-121.29043 38.71797)
3,CH1,N,2018-05-30,2018.00,16657,Auburn Boulevard,Sacramento,90099.00,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,95.07,310761794.00,502910719.00,POINT (-121.29040 38.70704)
4,CH1,S,2018-05-30,2018.00,None,Auburn Boulevard,Sacramento,90136.00,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,205.17,310761735.00,617672276.00,POINT (-121.29040 38.70569)


In [50]:
sc_stv['Long'] = sc_stv.geometry.x
sc_stv['Lat'] = sc_stv.geometry.y

In [51]:
sc_stv.shape

(5711, 175)

In [52]:
sc14=sc_stv.loc[sc_stv['year']==2014]
sc15=sc_stv.loc[sc_stv['year']==2015]
sc16=sc_stv.loc[sc_stv['year']==2016]
sc17=sc_stv.loc[sc_stv['year']==2017]
sc18=sc_stv.loc[sc_stv['year']==2018]
sc19=sc_stv.loc[sc_stv['year']==2019]

In [53]:
sc_stv.groupby(['year'])['ID'].count()

year
2014.00     554
2015.00     819
2016.00    1890
2017.00     754
2018.00    1253
2019.00     441
Name: ID, dtype: int64

In [54]:
sc_stv.shape

(5711, 175)

In [55]:
sc_stv.shape

(5711, 175)

In [56]:
# read 2019 strava data
d1d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2019_CA_State/all_edges_yearly_2019_ride_D1/59f453a1ea74ff48d88556f72ba70b289413e3c9988a49f55e3d722a06f0bd08-1651249687876.csv')
d2d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2019_CA_State/all_edges_yearly_2019_ride_D2/20893741ac1090f60daf98c1c5fbda5781de61ec0cfac405ac696082a3c3913c-1651249705530.csv')
d3d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2019_CA_State/all_edges_yearly_2019_ride_D3/dee63afb3a1adb1b662700f92d8c66eb58ee2dc702c98880c80ffe6b57983118-1651249721501.csv')
d4d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2019_CA_State/all_edges_yearly_2019_ride_D4/7e3ac4a93435f0ee9bd5ca4d3c0695a89d83613a09ae96bc14b4bc9cdaf2dbfc-1651518670665.csv')
d5d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2019_CA_State/all_edges_yearly_2019_ride_D5/ea61d5f36b0efe14dd85ab36756c8082f6f0bc38e7b2bd01d87da4cc9e997dce-1651518685821.csv')
d6d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2019_CA_State/all_edges_yearly_2019_ride_D6/343c5fd09a7e61d6226038b77ba41d9d673a2a2bb78e0c4efa0b8d4dc9e0c7f1-1651518708125.csv')
d7d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2019_CA_State/all_edges_yearly_2019_ride_D7/780268bacba82a74eb5a25991baa2656a5ee05bcb3830379d46b26cdf36bf1e5-1651606611912.csv')
d8d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2019_CA_State/all_edges_yearly_2019_ride_D8/2bc3c7efd5db2e5823f3f9b3f8af13f56e588adf635dcf29da5506ea49bacee4-1651606628564.csv')
d9d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2019_CA_State/all_edges_yearly_2019_ride_D9/752912d517b003fd103580e3d58726805be1ec9c264aff9401b41ba46be74c9e-1651606647084.csv')
d10d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2019_CA_State/all_edges_yearly_2019_ride_D10/b33a496702ca56dfbc68aa7f90aef3c4a1187de5fc1d6cc6b89b0fd0955847f0-1651682235585.csv')
d11d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2019_CA_State/all_edges_yearly_2019_ride_D11/9a0657f7164b7112f9a241c67e731e590b2a3c5b17216728b041e9476ae4ebea-1651682309481.csv')
d12d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2019_CA_State/all_edges_yearly_2019_ride_D12/b5679e9cff99ad7f276d6c7e480190bdc1ddb518fa1327066a9ab73ce1a76e51-1651682326989.csv')
# dCA=gpd.read_file('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2019_CA_State/Strava2019_CA_State/CA_Strava_Annual_Data_2019_v.shp')
# dCA.head()

In [57]:
# read Carson data
dCarson=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/selected_area_yearly_2019_ride_Carson/21e9984d65baea8ab9462f4b6460ea572b7a34c497cc9a082901ea345c44f6ee-1668731073066.csv')

In [58]:
CA_Strava_data19=pd.concat([d1d,d2d,d3d,d4d,d5d,d6d,d7d,d8d,d9d,d10d,d11d,d12d,dCarson],axis=0)
CA_Strava_data19.head()

,edge_uid,activity_type,year,forward_trip_count,reverse_trip_count,forward_people_count,reverse_people_count,forward_commute_trip_count,reverse_commute_trip_count,forward_leisure_trip_count,...,osm_reference_id,total_trip_count,forward_midday_trip_count,reverse_midday_trip_count,forward_overnight_trip_count,reverse_overnight_trip_count,forward_18_34_people_count,reverse_18_34_people_count,ride_count,ebike_ride_count
0,307087344,Ride,2019,0,5,0,5,0,0,0,...,10249524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,307093461,Ride,2019,160,255,35,100,10,40,150,...,10266924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,307093816,Ride,2019,125,115,15,10,95,85,30,...,396489943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,307094834,Ride,2019,15,10,10,5,0,0,15,...,10249947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,307095856,Ride,2019,75,70,35,15,0,5,75,...,136229365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
sc_stv19_data=pd.merge(sc19,CA_Strava_data19, left_on=['edgeUID','osmId','year'],right_on=['edge_uid','osm_reference_id','year'],how='inner')
sc_stv19_data['stv_total']=sc_stv19_data['forward_trip_count']+sc_stv19_data['reverse_trip_count']
sc_stv19_data['stv_com_total']=sc_stv19_data['forward_commute_trip_count']+sc_stv19_data['reverse_commute_trip_count']
sc_stv19_data['stv_rec_total']=sc_stv19_data['forward_leisure_trip_count']+sc_stv19_data['reverse_leisure_trip_count']
sc_stv19_data['stv_com_ratio']=sc_stv19_data['stv_com_total']/sc_stv19_data['stv_total']
sc_stv19_data['stv_rec_ratio']=sc_stv19_data['stv_rec_total']/sc_stv19_data['stv_total']
sc_stv19_data.shape

(384, 222)

In [60]:
# read 2018 data
# Read distrcit 1-12 Strava data
d1d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2018_CA_State/all_edges_yearly_2018_ride_D1/6ef13ecebbf2b7a8bd7bf242a08a1187e5e74ede8d1e2c1f40f21f9a71379e55-1650903378805.csv')
d2d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2018_CA_State/all_edges_yearly_2018_ride_D2/44cf2e03b96404b9872bf550eaea24f813317d0d5a817cfcea9fad696805a734-1650903428425.csv')
d3d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2018_CA_State/all_edges_yearly_2018_ride_D3/255a92b95d28169d37fc67c83e39e444e0643dfd3bfaf6fec1da60893e02a9e2-1650903448299.csv')
d4d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2018_CA_State/all_edges_yearly_2018_ride_D4/075a1d3c2bcaa8641098036f74342333afbdaa711e1ae3b7a5688f1c4f0eaf71-1650987233391.csv')
d5d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2018_CA_State/all_edges_yearly_2018_ride_D5/c866c5664e8223e5c800237e3565776dade2e0961603ae6c87310cbdbf8ba9af-1650987250914.csv')
d6d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2018_CA_State/all_edges_yearly_2018_ride_D6/a3feac8689d6c00d604c7609d7c7d7a8c6ddb34f847874598f718c8e8022ac17-1650987275159.csv')
d7d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2018_CA_State/all_edges_yearly_2018_ride_D7/ea612ab7e36f3e26b9be6c34fc9b78908045107a144f18e59653f45c3c5fafdf-1651076571753.csv')
d8d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2018_CA_State/all_edges_yearly_2018_ride_D8/f308fe3ecdf647a71041f0d90778d3489ed937495a9a303476a9f9fa273773c5-1651076594138.csv')
d9d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2018_CA_State/all_edges_yearly_2018_ride_D9/972916ce8ab7a5261a5dcc7dec4d61b5ba6b66173d4e67b5ec3e757deb6d07e4-1651076614829.csv')
d10d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2018_CA_State/all_edges_yearly_2018_ride_D10/800cbbe4f28cdcdf8ccc1cd656e6d709811118ff55be4f92a5b8fb4cc27d6008-1651163735851.csv')
d11d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2018_CA_State/all_edges_yearly_2018_ride_D11/292b5f0d8a1847f7076afa78c3d30319a742d0fdc1adbab22eea1b936cdba43c-1651163755994.csv')
d12d=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/Strava2018_CA_State/all_edges_yearly_2018_ride_D12/e92c8e0909e62306b6d616045980c526c280346cb41e1e6bc723f5bd63c66e44-1651163779808.csv')
dCarson=pd.read_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Strava/selected_area_yearly_2018_ride_carson_city/fc2eba658781af8b2c78e649da0705cd5e3087f149abed4474a15598bd6f5e9f-1668731100883.csv')
CA_Strava_data18=pd.concat([d1d,d2d,d3d,d4d,d5d,d6d,d7d,d8d,d9d,d10d,d11d,d12d,dCarson],axis=0)
CA_Strava_data18.head()

,edge_uid,activity_type,year,forward_trip_count,reverse_trip_count,forward_people_count,reverse_people_count,forward_commute_trip_count,reverse_commute_trip_count,forward_leisure_trip_count,...,osm_reference_id,total_trip_count,forward_midday_trip_count,reverse_midday_trip_count,forward_overnight_trip_count,reverse_overnight_trip_count,forward_18_34_people_count,reverse_18_34_people_count,ride_count,ebike_ride_count
0,307078653,Ride,2018,25,15,15,15,0,0,25,...,10306659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,307080403,Ride,2018,10,5,10,5,0,0,10,...,238818704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,307081105,Ride,2018,350,90,340,85,0,0,350,...,826580917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,307098773,Ride,2018,5,10,5,10,0,0,5,...,34576425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,307106597,Ride,2018,0,70,0,15,0,40,0,...,759146979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# Make average Strava data for 2014, 2015, 2016 and 2017 short term sites
stv18=CA_Strava_data18[['edge_uid','osm_reference_id','year','forward_trip_count','reverse_trip_count','forward_leisure_trip_count','reverse_leisure_trip_count','forward_commute_trip_count','reverse_commute_trip_count' ]]
stv19=sc_stv19_data[['edge_uid','osm_reference_id','year','forward_trip_count','reverse_trip_count','forward_leisure_trip_count','reverse_leisure_trip_count','forward_commute_trip_count','reverse_commute_trip_count' ]]
stv1819c=pd.concat([stv19,stv18],axis=0)
stv1819c['stv_total0']=stv1819c['forward_trip_count']+stv1819c['reverse_trip_count']
stv1819c['stv_com_total0']=stv1819c['forward_commute_trip_count']+stv1819c['reverse_commute_trip_count']
stv1819c['stv_rec_total0']=stv1819c['forward_leisure_trip_count']+stv1819c['reverse_leisure_trip_count']
# sc_stv1819['stv_com_ratio']=sc_stv1819['stv_com_total']/sc_stv1819['stv_total']
# sc_stv1819['stv_rec_ratio']=sc_stv1819['stv_rec_total']/sc_stv1819['stv_total']
stv_total_ave=pd.DataFrame({'stv_total':stv1819c.groupby(['edge_uid','osm_reference_id'])['stv_total0'].mean()}).reset_index()
stv_com_ave=pd.DataFrame({'stv_com_total':stv1819c.groupby(['edge_uid','osm_reference_id'])['stv_com_total0'].mean()}).reset_index()
stv_rec_ave=pd.DataFrame({'stv_rec_total':stv1819c.groupby(['edge_uid','osm_reference_id'])['stv_rec_total0'].mean()}).reset_index()
stv_rec_ave.head()

,edge_uid,osm_reference_id,stv_rec_total
0,305580685,173567461,110.00
1,305580686,173567461,110.00
2,305580687,173567463,110.00
3,305580688,355677914,65.00
4,305580689,691345389,70.00


In [62]:
stv_total_ave.shape, stv_com_ave.shape,stv_rec_ave.shape

((3353636, 3), (3353636, 3), (3353636, 3))

In [63]:
# merge the average value
stv1=pd.merge(stv_total_ave,stv_com_ave, on=['edge_uid','osm_reference_id'], how='inner')
stv2=pd.merge(stv1,stv_rec_ave, on=['edge_uid','osm_reference_id'], how='inner')
stv2['stv_rec_ratio']=stv2['stv_rec_total']/stv2['stv_total']
stv2.head()

,edge_uid,osm_reference_id,stv_total,stv_com_total,stv_rec_total,stv_rec_ratio
0,305580685,173567461,110.00,0.00,110.00,1.00
1,305580686,173567461,115.00,5.00,110.00,0.96
2,305580687,173567463,110.00,0.00,110.00,1.00
3,305580688,355677914,65.00,0.00,65.00,1.00
4,305580689,691345389,70.00,0.00,70.00,1.00


In [64]:
sc_stv18_data=pd.merge(sc18,CA_Strava_data18, left_on=['edgeUID','osmId','year'],right_on=['edge_uid','osm_reference_id','year'],how='inner')
sc_stv18_data['stv_total']=sc_stv18_data['forward_trip_count']+sc_stv18_data['reverse_trip_count']
sc_stv18_data['stv_com_total']=sc_stv18_data['forward_commute_trip_count']+sc_stv18_data['reverse_commute_trip_count']
sc_stv18_data['stv_rec_total']=sc_stv18_data['forward_leisure_trip_count']+sc_stv18_data['reverse_leisure_trip_count']
sc_stv18_data['stv_com_ratio']=sc_stv18_data['stv_com_total']/sc_stv18_data['stv_total']
sc_stv18_data['stv_rec_ratio']=sc_stv18_data['stv_rec_total']/sc_stv18_data['stv_total']
sc_stv18_data.shape

(1169, 222)

In [65]:
sc_stv14_data=pd.merge(sc14,stv2, left_on=['edgeUID','osmId'],right_on=['edge_uid','osm_reference_id'],how='inner')
sc_stv14_data.shape

(528, 181)

In [66]:
sc_stv15_data=pd.merge(sc15,stv2, left_on=['edgeUID','osmId'],right_on=['edge_uid','osm_reference_id'],how='inner')
sc_stv15_data.shape

(795, 181)

In [67]:
sc_stv16_data=pd.merge(sc16,stv2, left_on=['edgeUID','osmId'],right_on=['edge_uid','osm_reference_id'],how='inner')
sc_stv16_data.shape

(1829, 181)

In [68]:
sc_stv17_data=pd.merge(sc17,stv2, left_on=['edgeUID','osmId'],right_on=['edge_uid','osm_reference_id'],how='inner')
sc_stv17_data.head()

,ID,Leg,date_1,year,segment_id,street_nam,county,tdg_id,lrs_cal_id,bikes_proh,...,osmId,geometry,Long,Lat,edge_uid,osm_reference_id,stv_total,stv_com_total,stv_rec_total,stv_rec_ratio
0,Kern6,N,2017-11-09,2017.00,14352,Fairfax Road,Kern,28765.00,0,0,...,395967189.00,POINT (-118.93202 35.35562),-118.93,35.36,308518382,395967189,195.00,32.50,162.50,0.83
1,Kern74,N,2017-02-15,2017.00,14384,Eissler Street,Kern,28795.00,0,0,...,10442962.00,POINT (-118.93873 35.39975),-118.94,35.40,308524926,10442962,10.00,0.00,10.00,1.00
2,Kern77,W,2017-04-27,2017.00,13043,Norris Road,Kern,27584.00,0,0,...,395791229.00,POINT (-119.02617 35.41953),-119.03,35.42,308508296,395791229,255.00,30.00,225.00,0.88
3,Kern78,N,2017-04-27,2017.00,13102,McCray Street,Kern,27642.00,0,0,...,10407487.00,POINT (-119.03007 35.42001),-119.03,35.42,308512288,10407487,42.50,7.50,35.00,0.82
4,Kern79,E,2017-04-27,2017.00,13123,Norris Road,Kern,27660.00,0,0,...,395791229.00,POINT (-119.03294 35.41955),-119.03,35.42,308508231,395791229,225.00,30.00,195.00,0.87


In [69]:
sc16.shape,sc17.shape

((1890, 175), (754, 175))

In [70]:
sc_stv=pd.concat([sc_stv14_data,sc_stv15_data,sc_stv16_data,sc_stv17_data,sc_stv18_data,sc_stv19_data],axis=0)
sc_stv.shape

(5374, 222)

In [71]:
sc17.shape, sc16.shape

((754, 175), (1890, 175))

In [72]:
sc1914=pd.concat([sc19,sc18,sc17,sc16,sc15,sc14],axis=0)

In [73]:
# figure out the short term sites where strava activities zero
common = sc1914.merge(sc_stv,on=['ID', 'year', 'Leg'])
sc_stv1914r=sc1914[(~sc1914.ID.isin(common.ID))]
sc_stv1914r.shape

(166, 175)

In [74]:
sc_stv.columns

Index(['ID', 'Leg', 'date_1', 'year', 'segment_id', 'street_nam', 'county',
       'tdg_id', 'lrs_cal_id', 'bikes_proh',
       ...
       'total_trip_count', 'forward_midday_trip_count',
       'reverse_midday_trip_count', 'forward_overnight_trip_count',
       'reverse_overnight_trip_count', 'forward_18_34_people_count',
       'reverse_18_34_people_count', 'ride_count', 'ebike_ride_count',
       'stv_com_ratio'],
      dtype='object', length=222)

In [75]:
sc_stv_v1=sc_stv[['ID', 'Leg','year', 'Lat', 'Long', 'edgeUID', 'osmId', 
       'stv_total', 'stv_com_total', 'stv_rec_total', 'stv_com_ratio',
       'stv_rec_ratio']]
sc_stv1419v1r1=sc_stv1914r[['ID', 'Leg','year', 'Lat', 'Long', 'edgeUID', 'osmId']]


In [77]:
sc_stv_v1['ID'].nunique(),sc_stv1419v1r1['ID'].count()

(2190, 166)

In [78]:
sc_stv_all=pd.concat([sc_stv_v1,sc_stv1419v1r1],axis=0)
sc_stv_all['stv_rec_ratio']=sc_stv_all['stv_rec_ratio'].fillna(0)
sc_stv_all.shape

(5540, 12)

In [79]:
sc_stv_all.head()

,ID,Leg,year,Lat,Long,edgeUID,osmId,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio
0,Eric_Fischer1089,N,2014.00,34.06,-118.31,307766980.00,47895363.00,3905.00,865.00,3040.00,NaN,0.78
1,Eric_Fischer1089,E,2014.00,34.06,-118.31,307766746.00,398048051.00,4970.00,1885.00,3085.00,NaN,0.62
2,Eric_Fischer1089,S,2014.00,34.06,-118.31,307766737.00,398053760.00,2050.00,620.00,1430.00,NaN,0.70
3,Eric_Fischer1089,W,2014.00,34.06,-118.31,307766743.00,398048057.00,3560.00,1830.00,1730.00,NaN,0.49
4,Eric_Fischer1086,E,2014.00,34.09,-118.31,307771629.00,398048112.00,2390.00,905.00,1485.00,NaN,0.62


In [80]:
sc_stv_all['stv_rec_ratio'].isnull().sum()

0

In [81]:
sc_stv_all.to_csv('C:/UC Berkeley Project Backup/Bike_ped Exposure Model Project/Processed Data/sc_stv_2014_2019.csv')

# end of the code